## Githun 연동하기

## 라이브러리 불러오기

In [1]:
# API 호출
# ## https://data.go.kr/tcs/dss/selectApiDataDetailView.do?publicDataPk=15057440
from urllib.request import Request, urlopen
# Request https://docs.python.org/ko/3/library/urllib.request.html#urllib.request.Request
# urlopen https://docs.python.org/ko/3/library/urllib.request.html#urllib.request.urlopen
from urllib.parse import urlencode, quote_plus
# urlenocde https://docs.python.org/ko/3/library/urllib.parse.html#urllib.parse.urlencode
# quote_plus https://docs.python.org/ko/3/library/urllib.parse.html#urllib.parse.quote_plus
from urllib.parse import unquote
import requests

# 시간 생성
import time
import datetime
from pytz import timezone, utc

# 폴더 자동 생성
import os

In [2]:
# 폴더 자동 생성 함수
# https://data-make.tistory.com/170
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)

## KEY unquote

In [3]:
KEY = 'yEaR%2F3MDedRSlVJL%2F2pxnVg0yre1N5VF3RZ%2FUAt56MJ7J2mNpfqhUvy05pXV0uhHTVY7DbyCR8xmMaDdYga67Q%3D%3D' # 종현
# apiKEY = requests.utils.unquote(KEY)
apiKey = unquote(KEY)
# print(apiKey)
KEYSW = 'M%2B4%2FqUiadT8X8PhgFjaQLDu%2BIOgPMURfGsOX%2FmVxwHQVJgnVR%2FMPjDYXkuQNwUFbZXlfnX5Lls3SUCiCLIFjgQ%3D%3D'
apiKeySW = unquote(KEYSW)

## URL 생성

### ws.bus.go.kr url을 사용하는 경우 
1.   'ServiceKey' 대문자 S
2.   'busRouteId' 노선ID는 busRouteId

In [4]:
# getArrInfoByRouteAll
url = 'http://ws.bus.go.kr/api/rest/arrive/getArrInfoByRouteAll'
queryParams = '?' + urlencode({ quote_plus('ServiceKey') : apiKey, quote_plus('busRouteId') : '204000046' })

In [5]:
# getBusPosByRtid
# 노선ID로 차량들의 위치정보를 조회합니다
url = 'http://ws.bus.go.kr/api/rest/buspos/getBusPosByRtid'
queryParams = '?' + urlencode({ quote_plus('ServiceKey') : apiKey, quote_plus('busRouteId') : '204000046' })

In [6]:
url+queryParams

'http://ws.bus.go.kr/api/rest/buspos/getBusPosByRtid?ServiceKey=yEaR%2F3MDedRSlVJL%2F2pxnVg0yre1N5VF3RZ%2FUAt56MJ7J2mNpfqhUvy05pXV0uhHTVY7DbyCR8xmMaDdYga67Q%3D%3D&busRouteId=204000046'

### openapi url을 사용하는 경우 
1.   'serviceKey' 소문자 s
2.   'routeId' 노선ID는 routeId

In [7]:
# 노선번호목록조회 busrouteservice
# 해당 노선번호의 노선ID, 노선유형, 운행지역 등을 제공합니다.
url = 'http://openapi.gbis.go.kr/ws/rest/busrouteservice' 
queryParams = '?' + urlencode({ quote_plus('serviceKey') : apiKey, quote_plus('keyword') : '9000' })

In [8]:
# 노선정보항목조회 busrouteservice/info
# 해당 노선에 대한 노선번호, 기점/종점 정류소, 첫차/막차시간, 배차간격, 운행업체 등의 운행계획 정보를 제공합니다.
url = 'http://openapi.gbis.go.kr/ws/rest/busrouteservice/info'
queryParams = '?' + urlencode({ quote_plus('serviceKey') : apiKey, quote_plus('routeId') : '204000046' })

In [9]:
# 경유정류소목록조회 busrouteservice/info
# 해당 노선이 정차하는 경유정류소 목록과 정류소명, 중앙차로여부, 회차점, 좌표값 등을 제공합니다.
url = 'http://openapi.gbis.go.kr/ws/rest/busrouteservice/station'
queryParams = '?' + urlencode({ quote_plus('serviceKey') : apiKey, quote_plus('routeId') : '204000046' })

In [10]:
# 경유정류소목록조회 buslocationservice
# 해당 노선이 정차하는 경유정류소 목록과 정류소명, 중앙차로여부, 회차점, 좌표값 등을 제공합니다.
url = 'http://openapi.gbis.go.kr/ws/rest/buslocationservice'
queryParams = '?' + urlencode({ quote_plus('serviceKey') : apiKey, quote_plus('routeId') : '204000046' })

## routeIdList를 이용하여 여러 노선에 대해 호출

In [11]:
routeIdList = \
[
234000013,
234000130,
204000056,
204000046,
204000041,
204000070,
204000059,
204000060,
204000022,
204000039,
204000040,
204000067,
204000044,
204000147,
]

## filePath 생성

In [12]:
fileAPIName = 'busAPITestOutput'

for routeId in routeIdList:

    # 코드를 실행하는 장치에 따라 달라지는 경로 입니다. 수정해서 사용하세요.
    fileDevicepath = 'C:/Users/jongh/OneDrive/School/석사/1-2/자료구조/프로젝트/5.DATA'
    # yourDevicepath = 'C:/Users'

    # 고정된 이름 및 변수에 의해 자동으로 생성 되는 이름 입니다. 변수를 확인하세요.
    fileRootPath = '/Master/dataAPI/'
    fileAPIName = fileAPIName
    fileRouteId = str(routeId)                                                                                   # 파일 이름에 routeId를 포함 시킬지 확인합니다.
    fileDateTime = utc.localize(datetime.datetime.utcnow()).astimezone(timezone('Asia/Seoul')).isoformat()       # 자정이 넘어간 시간 파일 경로가 바뀌는 문제를 해결해야 합니다.
    fileDateStr = fileDateTime[2:10]
    fileExtension = '.txt'

    #파일의 폴더 경로와 이름을 각각 출력합니다.
    folderPath = str(fileDevicepath + fileRootPath + fileAPIName + '/' + fileRouteId)
    # print(folderPath)
    fileName = fileRouteId + '_' + fileDateStr + fileExtension
    # print(fileName)

    # 파일의 최종 경로를 출력합니다.
    filePath = str(folderPath + '/' + fileName)
    
    # 폴더를 생성합니다.
    createFolder(folderPath)
    
    print(filePath)

C:/Users/jongh/OneDrive/School/석사/1-2/자료구조/프로젝트/5.DATA/Master/dataAPI/busAPITestOutput/234000013/234000013_20-11-10.txt
C:/Users/jongh/OneDrive/School/석사/1-2/자료구조/프로젝트/5.DATA/Master/dataAPI/busAPITestOutput/234000130/234000130_20-11-10.txt
C:/Users/jongh/OneDrive/School/석사/1-2/자료구조/프로젝트/5.DATA/Master/dataAPI/busAPITestOutput/204000056/204000056_20-11-10.txt
C:/Users/jongh/OneDrive/School/석사/1-2/자료구조/프로젝트/5.DATA/Master/dataAPI/busAPITestOutput/204000046/204000046_20-11-10.txt
C:/Users/jongh/OneDrive/School/석사/1-2/자료구조/프로젝트/5.DATA/Master/dataAPI/busAPITestOutput/204000041/204000041_20-11-10.txt
C:/Users/jongh/OneDrive/School/석사/1-2/자료구조/프로젝트/5.DATA/Master/dataAPI/busAPITestOutput/204000070/204000070_20-11-10.txt
C:/Users/jongh/OneDrive/School/석사/1-2/자료구조/프로젝트/5.DATA/Master/dataAPI/busAPITestOutput/204000059/204000059_20-11-10.txt
C:/Users/jongh/OneDrive/School/석사/1-2/자료구조/프로젝트/5.DATA/Master/dataAPI/busAPITestOutput/204000060/204000060_20-11-10.txt
C:/Users/jongh/OneDrive/School/석사/1-2/자료

## 폴더 생성하기

In [13]:
#createFolder(folderPath)

## API 호출

In [15]:
# {routeID : filePath}
textFile = open(filePath,                     # 파일 경로가 routeId에 맞게 바뀌는지 확인합니다. >>> 바뀌지 않음: 코드 수정 필요 합니다.
                'w',                          # 파일을 덮어 쓰고 있지 않은지 확인합니다. >>> 덮어쓰고 있음: 코드 수정이 필요한지 확인합니다.
                encoding = 'utf-8',
                newline = '\n')
url = 'http://openapi.gbis.go.kr/ws/rest/buslocationservice'
for i in range(10):
    for routeId in routeIdList:
        queryParams = '?' + urlencode({ quote_plus('serviceKey') : apiKey, quote_plus('routeId') : routeId })
        request = Request(url + queryParams)
        request.get_method = lambda: 'GET'
        oneLineXML = urlopen(request).read().decode('utf8')
        textFile.write(oneLineXML)
        textFile.write('\n')
    print(fileDateTime[11:19],'\t', i+1, '번째 반복문을 실행하였습니다.')
textFile.close()

19:34:52 	 1 번째 반복문을 실행하였습니다.
19:34:52 	 2 번째 반복문을 실행하였습니다.
19:34:52 	 3 번째 반복문을 실행하였습니다.
19:34:52 	 4 번째 반복문을 실행하였습니다.
19:34:52 	 5 번째 반복문을 실행하였습니다.
19:34:52 	 6 번째 반복문을 실행하였습니다.
19:34:52 	 7 번째 반복문을 실행하였습니다.
19:34:52 	 8 번째 반복문을 실행하였습니다.
19:34:52 	 9 번째 반복문을 실행하였습니다.
19:34:52 	 10 번째 반복문을 실행하였습니다.
